In [8]:
from google.colab import drive
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
import tensorflow as tf

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def process_data(data, headers):
    for header_idx, header in enumerate(headers):
        if header == "school":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "GP":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif header == "sex":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "F":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif header == "address":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "U":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif header == "famsize":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "LE3":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif header == "Pstatus":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "T":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif header == "Mjob" or header == "Fjob":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "teacher":
                    data[row_idx, header_idx] = 0
                elif data[row_idx, header_idx] == "health":
                    data[row_idx, header_idx] = 1
                elif data[row_idx, header_idx] == "services":
                    data[row_idx, header_idx] = 2
                elif data[row_idx, header_idx] == "at_home":
                    data[row_idx, header_idx] = 3
                else:
                    data[row_idx, header_idx] = 4
        elif header == "reason":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "home":
                    data[row_idx, header_idx] = 0
                elif data[row_idx, header_idx] == "reputation":
                    data[row_idx, header_idx] = 1
                elif data[row_idx, header_idx] == "course":
                    data[row_idx, header_idx] = 2
                else:
                    data[row_idx, header_idx] = 3
        elif header == "guardian":
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "mother":
                    data[row_idx, header_idx] = 0
                elif data[row_idx, header_idx] == "father":
                    data[row_idx, header_idx] = 1
                else:
                    data[row_idx, header_idx] = 2
        elif (header == "schoolsup" or header == "famsup" or header == "paid"
            or header == 'activities' or header == 'nursery' or header == 'higher'
            or header == 'higher' or header == 'internet' or header == 'romantic'):
            for row_idx in range(len(data[0:, header_idx])):
                if data[row_idx, header_idx] == "yes":
                    data[row_idx, header_idx] = 0
                else:
                    data[row_idx, header_idx] = 1
        elif (header == "G1" or header == "G2" or header == "G3"):
          for row_idx in range(len(data[0:, header_idx])):
            data[row_idx, header_idx] = int(int(data[row_idx, header_idx]) >= 10)
    return data.astype(int)

def get_data():
    with open('/content/drive/My Drive/Colab Notebooks/proj/student-por.csv') as f:
        por_data = np.genfromtxt(f, delimiter=';', dtype='unicode')  

    for row_idx in range(len(por_data)):
        for col_idx in range(len(por_data[row_idx])):
            por_data[row_idx, col_idx] = por_data[row_idx, col_idx].strip('"')
    
    por_headers = por_data[0,:]
    por_data = por_data[1:, :]

    with open('/content/drive/My Drive/Colab Notebooks/proj/student-mat.csv') as f:
        mat_data = np.genfromtxt(f, delimiter=';', dtype='unicode')  

    for row_idx in range(len(mat_data)):
        for col_idx in range(len(mat_data[row_idx])):
            mat_data[row_idx, col_idx] = mat_data[row_idx, col_idx].strip('"')
    mat_headers = mat_data[0,:]
    mat_data = mat_data[1:, :]

    return ((mat_headers,mat_data),(por_headers,por_data))

In [10]:
# read in data from csvs
(mat_headers, mat_data), (por_headers, por_data) = get_data()

# reformat data to all integers
por_data = process_data(por_data, por_headers)
mat_data = process_data(mat_data, mat_headers)

por_x = por_data[:, :-3]
por_g1 = por_data[:, -3]
por_g2 = por_data[:, -2]
por_g3 = por_data[:, -1]

mat_x = mat_data[:, :-3]
mat_g1 = mat_data[:, -3]
mat_g2 = mat_data[:, -2]
mat_g3 = mat_data[:, -1]

# Define Model

In [49]:
def get_dnn (X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y, shuffle = True, random_state=2)
  dnn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, input_dim=X_train.shape[1], activation = 'relu'),
    tf.keras.layers.Dense(10, activation='relu'),      
    tf.keras.layers.Dense(10, activation='relu'),      
    tf.keras.layers.Dense(10, activation='relu'),           
    tf.keras.layers.Dense(1, activation='sigmoid')])

  dnn.compile(loss='BinaryCrossentropy', optimizer='SGD', metrics=['accuracy'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
  dnn.fit(X_train, y_train, epochs=150, batch_size=1, callbacks=[callback])
  score = dnn.evaluate(X_test, y_test, verbose=0)
  return score[1]

(649,)


# Perform analysis

In [50]:
por_g1_acc = get_dnn(por_x, por_g1)
por_g2_acc = get_dnn(por_x, por_g2)
por_g3_acc = get_dnn(por_x, por_g3)

mat_g1_acc = get_dnn(mat_x, mat_g1)
mat_g2_acc = get_dnn(mat_x, mat_g2)
mat_g3_acc = get_dnn(mat_x, mat_g3)

Epoch 1/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5968 - accuracy: 0.7511
Epoch 2/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5666 - accuracy: 0.7577
Epoch 3/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7577
Epoch 4/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7577
Epoch 5/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5344 - accuracy: 0.7577
Epoch 6/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5205 - accuracy: 0.7577
Epoch 7/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5034 - accuracy: 0.7577
Epoch 1/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7643
Epoch 2/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5453 - accuracy: 0.7731
Epoch 3/150
454/454 [==============================] - 1s 2ms/step - loss: 0.5284 - accurac

# Portugese Accuracy

In [51]:
print(f"G1: {por_g1_acc}%")
print(f"G2: {por_g2_acc}%")
print(f"G3: {por_g3_acc}%")

G1: 0.7692307829856873%
G2: 0.7743589878082275%
G3: 0.8461538553237915%


# Mathematics Accuracy

In [52]:
print(f"G1: {mat_g1_acc}%")
print(f"G2: {mat_g2_acc}%")
print(f"G3: {mat_g3_acc}%")

G1: 0.6974790096282959%
G2: 0.6974790096282959%
G3: 0.6638655662536621%
